In [1]:
import time
import random
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def KaliMatriks(A, B, n):
    """
    Mengalikan matriks A dan B yang berukuran n × n,
    menghasilkan matriks C yang juga berukuran n × n
    """
    # Deklarasi i, j, k : integer
    # Inisialisasi C sebagai matriks n×n berisi nol
    C = [[0 for j in range(n)] for i in range(n)]

    # Algoritma:
    for i in range(n):                  # for i ← 1 to n do
        for j in range(n):              #   for j ← 1 to n do
            C[i][j] = 0                 #     C[i, j] ← 0 {inisialisasi penjumlah}
            for k in range(n):          #     for k ← 1 to n do
                C[i][j] = C[i][j] + A[i][k] * B[k][j]
            #     endfor
        #   endfor
    # endfor

    return C


In [3]:
def add_matrix_count(A, B, counter):
    n = len(A)
    C = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            counter['count'] += 1
            C[i][j] = A[i][j] + B[i][j]
    return C

def sub_matrix_count(A, B, counter):
    n = len(A)
    C = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            counter['count'] += 1
            C[i][j] = A[i][j] - B[i][j]
    return C

def KaliMatriksStrassen_count(A, B, n, counter):
    """Strassen multiplication with counting operations."""
    if n == 1:
        counter['count'] += 1  # one scalar multiplication
        return [[A[0][0] * B[0][0]]]
    mid = n // 2
    A11 = [[A[i][j] for j in range(mid)] for i in range(mid)]
    A12 = [[A[i][j] for j in range(mid, n)] for i in range(mid)]
    A21 = [[A[i][j] for j in range(mid)] for i in range(mid, n)]
    A22 = [[A[i][j] for j in range(mid, n)] for i in range(mid, n)]
    B11 = [[B[i][j] for j in range(mid)] for i in range(mid)]
    B12 = [[B[i][j] for j in range(mid, n)] for i in range(mid)]
    B21 = [[B[i][j] for j in range(mid)] for i in range(mid, n)]
    B22 = [[B[i][j] for j in range(mid, n)] for i in range(mid, n)]

    M1 = KaliMatriksStrassen_count(sub_matrix_count(A12, A22, counter), add_matrix_count(B21, B22, counter), mid, counter)
    M2 = KaliMatriksStrassen_count(add_matrix_count(A11, A22, counter), add_matrix_count(B11, B22, counter), mid, counter)
    M3 = KaliMatriksStrassen_count(sub_matrix_count(A11, A21, counter), add_matrix_count(B11, B12, counter), mid, counter)
    M4 = KaliMatriksStrassen_count(add_matrix_count(A11, A12, counter), B22, mid, counter)
    M5 = KaliMatriksStrassen_count(A11, sub_matrix_count(B12, B22, counter), mid, counter)
    M6 = KaliMatriksStrassen_count(A22, sub_matrix_count(B21, B11, counter), mid, counter)
    M7 = KaliMatriksStrassen_count(add_matrix_count(A21, A22, counter), B11, mid, counter)

    C11 = add_matrix_count(sub_matrix_count(add_matrix_count(M1, M2, counter), M4, counter), M6, counter)
    C12 = add_matrix_count(M4, M5, counter)
    C21 = add_matrix_count(M6, M7, counter)
    temp = add_matrix_count(sub_matrix_count(M2, M3, counter), M5, counter)
    C22 = sub_matrix_count(temp, M7, counter)

    C = [[0] * n for _ in range(n)]
    for i in range(mid):
        for j in range(mid):
            C[i][j] = C11[i][j]
            C[i][j+mid] = C12[i][j]
            C[i+mid][j] = C21[i][j]
            C[i+mid][j+mid] = C22[i][j]
    return C

In [4]:
def KaliMatriks_count(A, B, n, counter):
    """Brute-force multiplication with operation counting."""
    C = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            for k in range(n):
                # One multiplication and one addition per loop
                counter['count'] += 2
                C[i][j] += A[i][k] * B[k][j]
    return C

In [ ]:
# Measurement
n_values = [2, 4, 8, 16, 32, 64, 512, 1024, 2048, 4096]
results_time = []
results_ops = []

for n in n_values:
    # Generate random matrices
    A = [[random.randint(0, 9999) for _ in range(n)] for _ in range(n)]
    B = [[random.randint(0, 9999) for _ in range(n)] for _ in range(n)]

    # Measure brute-force time
    start = time.perf_counter()
    KaliMatriks(A, B, n)
    t_bf = time.perf_counter() - start

    # Measure Strassen time
    start = time.perf_counter()
    KaliMatriksStrassen_count(A, B, n, {'count': 0})
    t_str = time.perf_counter() - start

    # Measure operations
    counter_bf = {'count': 0}
    KaliMatriks_count(A, B, n, counter_bf)
    counter_str = {'count': 0}
    KaliMatriksStrassen_count(A, B, n, counter_str)

    results_time.append({'n': n, 'BruteForce(s)': t_bf, 'Strassen(s)': t_str})
    results_ops.append({'n': n, 'BruteForce': counter_bf['count'], 'Strassen': counter_str['count']})

# Create DataFrames
df_time = pd.DataFrame(results_time)
df_ops = pd.DataFrame(results_ops)

# # Show tables
# import ace_tools as tools; tools.display_dataframe_to_user(name="Waktu Elapsed vs n", dataframe=df_time)
# import ace_tools as tools; tools.display_dataframe_to_user(name="Jumlah Operasi vs n", dataframe=df_ops)

# Plot Time vs n
plt.figure()
plt.plot(df_time['n'], df_time['BruteForce(s)'], label='Brute Force')
plt.plot(df_time['n'], df_time['Strassen(s)'], label='Strassen')
plt.xlabel('Ukuran n')
plt.ylabel('Waktu (detik)')
plt.title('Perbandingan Waktu Eksekusi')
plt.legend()
plt.show()

# Plot Ops vs n
plt.figure()
plt.plot(df_ops['n'], df_ops['BruteForce'], label='Brute Force')
plt.plot(df_ops['n'], df_ops['Strassen'], label='Strassen')
plt.xlabel('Ukuran n')
plt.ylabel('Jumlah Operasi')
plt.title('Perbandingan Kompleksitas Operasi')
plt.legend()
plt.show()